In [2]:
#import libraries
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import os

print(tf.__version__)
print(keras.__version__)

2.0.0-beta1
2.2.4-tf


In [3]:
#초모수 값 지정
learning_rate = 0.001
training_epochs = 15
batch_size = 100

tf.random.set_seed(787)

In [4]:
## MNIST Dataset #########################################################
mnist = keras.datasets.mnist
class_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
##########################################################################


In [5]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()    
    
train_images = train_images.astype(np.float32) / 255.
test_images = test_images.astype(np.float32) / 255.
train_images = np.expand_dims(train_images, axis=-1)
test_images = np.expand_dims(test_images, axis=-1)
print(len(test_images),len(train_labels))
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)    
    
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(
                buffer_size=10000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(len(test_images))

W0629 19:31:36.402994  8328 deprecation.py:323] From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\data\util\random_seed.py:58: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


10000 60000


## 모델 형성
#### 모델 클래스 상속 - fully customizable model 구현
tf.keras.Model 클래스를 상속하고 자신만의 정방향 패스(forward pass)을 정의하여 완전히 커스터마이징된 모델을 만들 수 있습니다. __init__ 메서드에서 층을 만들어 클래스 객체의 속성으로 지정합니다. 정방향 패스는 call 메서드에 정의합니다.

즉시 실행이 활성화되어 있을 때 정방향 패스를 명령형 프로그래밍 방식으로 작성할 수 있기 때문에 모델 클래스 상속이 매우 유용합니다.

노트: 정방향 패스를 항상 명령형 프로그래밍 방식으로 실행하려면 super 객체의 생성자를 호출할 때 dynamic=True를 지정하세요.

중요 포인트: 작업에 맞는 API를 사용하세요. 모델 클래스 상속은 유연성을 제공하지만 복잡도가 증가하고 사용자 오류가 발생할 가능성이 높아집니다. 가능한한 함수형 API를 사용하세요.
#### 참고 : 맞춤형 층 https://www.tensorflow.org/beta/guide/keras/overview
맞춤형 층(custom layer)을 만들려면 tf.keras.layers.Layer 클래스를 상속하고 다음 메서드를 구현합니다:

__init__: 이 층에서 사용되는 하위 층을 정의할 수 있습니다.
build: 층의 가중치를 만듭니다. add_weight 메서드를 사용해 가중치를 추가합니다.
call: 정방향 패스를 구현합니다.

In [22]:
from tensorflow.keras import layers 
class MyModel(tf.keras.Model):

  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1=keras.layers.Conv2D(filters=32, kernel_size=[3,3],padding='SAME',activation='relu')
    self.pool1=keras.layers.MaxPool2D(padding='SAME')
    self.conv2=keras.layers.Conv2D(filters=64, kernel_size=[3,3],padding='SAME',activation='relu')
    self.pool2=keras.layers.MaxPool2D(padding='SAME')    
    self.conv3=keras.layers.Conv2D(filters=128, kernel_size=[3,3],padding='SAME',activation='relu')    
    self.pool3=keras.layers.MaxPool2D(padding='SAME')
    self.flat=keras.layers.Flatten()
    self.dense_1 = layers.Dense(units=256, activation='relu')
    self.drop1=keras.layers.Dropout(rate=0.4)
    self.dense_2 = keras.layers.Dense(units=10, activation='softmax')
#from tensorflow.keras import layers 해두면 keras.lyers.~말고 걍 layes.~~ 해도됨

  def call(self, inputs,training=False):
    # 정방향 패스를 정의합니다.
    # `__init__` 메서드에서 정의한 층을 사용합니다.
    x =self.conv1(inputs)
    x=self.pool1(x)
    x =self.conv2(x)    
    x=self.pool2(x)
    x =self.conv3(x) 
    x=self.pool3(x)
    x =self.flat(x) 
    x=self.dense_1(x)
    x=self.drop1(x)
    x=self.dense_2(x)

    return x

In [23]:
model = MyModel()
temp_inputs=keras.Input(shape=(28,28,1))
model(inputs=temp_inputs)
model.summary()

Model: "my_model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 7, 7, 128)         73856     
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 4, 4, 128)         0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 2048)              0

In [34]:
import pydot
from tensorflow.keras.utils.vis_utils import plot_model

ModuleNotFoundError: No module named 'tensorflow.keras.utils.vis_utils'

In [30]:
import graphviz

In [35]:
tf.keras.utils.plot_model(model, 'my_first_model.png')

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


In [25]:
model.compile(optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy'])

In [26]:
#for checkpoint
# 파일 이름에 에포크 번호를 포함시킵니다(`str.format` 포맷)
checkpoint_path = "trainving_11_3/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 체크포인트 콜백 만들기
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
model.fit(train_dataset, epochs =2,
          validation_data =test_dataset,
          callbacks = [cp_callback])  # 훈련 단계에 콜백을 전달합니다

# 옵티마이저의 상태를 저장하는 것과 관련되어 경고가 발생할 수 있습니다.
# 이 경고는 (그리고 이 노트북의 다른 비슷한 경고는) 이전 사용 방식을 권장하지 않기 위함이며 무시해도 좋습니다.

Epoch 1/2
599/600 [============================>.] - ETA: 0s - loss: 0.1956 - accuracy: 0.9392
Epoch 00001: saving model to trainving_11_3/cp-0001.ckpt
600/600 [==============================] - 249s 415ms/step - loss: 0.1954 - accuracy: 0.9394 - val_loss: 0.0440 - val_accuracy: 0.9844
Epoch 2/2
599/600 [============================>.] - ETA: 0s - loss: 0.0569 - accuracy: 0.9825
Epoch 00002: saving model to trainving_11_3/cp-0002.ckpt
600/600 [==============================] - 230s 383ms/step - loss: 0.0568 - accuracy: 0.9825 - val_loss: 0.0319 - val_accuracy: 0.9900
